In [1]:
import sys
sys.path.append("../")
from utils import *

In [2]:
# Load in the data
d = {'pl':pd.read_csv(data_path+'pl_tracks.csv'),\
     't':pd.read_csv(data_path+'tracks.csv')}

# Calculate popularity: number of occurances
pop = d['pl'].groupby('track_uri').count().reset_index()
pop.columns = ['track_uri', 'popularity']
df = pd.merge(d['t'], pop, on = 'track_uri', how = 'left')

# Define the list of track_uri ordered by popularity
pop_sorted_tracks = df.sort_values('popularity', ascending = False)['track_uri'].values.tolist()
print(len(pop_sorted_tracks))
pop_sorted_tracks = pop_sorted_tracks[:(len(pop_sorted_tracks)//100)]
print(len(pop_sorted_tracks))

2262292
22622


In [3]:
del d
gc.collect()

49

In [4]:
with open(data_path + "train_test_data.pkl", "rb") as f:
    X_train, y_train, X_test, y_test = pickle.load(f)

# Define Similarity Metrics

In [5]:
def overlap_similarity(pl1, pl2):
    ''' We define the similarity between playlists to be:
        # overlap songs / min(len(pl1), len(pl2))'''
    overlap = sum([a in pl2 for a in pl1])
    return overlap / min(len(pl1), len(pl2))

In [6]:
def calc_similarity_map(X, similarity_fn, prefix, save = True):
    ''' Calcualte the pair-wise similarity between all keys in X, using similarity_fn. '''
    all_keys = list(X.keys())
    similarity_dict = {(a,b):np.inf for a in all_keys for b in all_keys}
    for i in range(len(all_keys)):
        if i % (len(all_keys)//10) == 0: print(i)
        for j in range(i, len(all_keys)):
            this_pl = all_keys[i]
            other_pl = all_keys[j]
            dist = similarity_fn(X[this_pl], X[other_pl])
            similarity_dict[(this_pl, other_pl)] = dist
            similarity_dict[(other_pl, this_pl)] = dist
    if save:
        with open(data_path + prefix + "_similarity_map.pkl", "wb") as f:
            pickle.dump(similarity_dict, f)

def load_similarity_map(prefix = 'X_train'):
    ''' Loads a pre-calculated similarity map '''
    with open(data_path + prefix + "_similarity_map.pkl", "rb") as f:
        similarity_dict = pickle.load(f)
    return similarity_dict

# Train KNN Ranking Model

In [7]:
# Calculate the similarity map for X_train
# calc_similarity_map(X_train, overlap_similarity, prefix ='X_train')
similarity_dict = load_similarity_map(prefix ='X_train')

In [8]:
def get_neighbor_dict(X, similarity_dict, n_neighbors = 100):
    ''' Use the input X to find the n_neighbors neighbors for each playlist using 
        precalculated similaritys from similarity_dict. '''
    all_keys = X.keys()
    neighbor_dict = {k:[] for k in all_keys}
    neighbor_sim_dict = {k:[] for k in all_keys}
    for i in range(len(all_keys)):
        this_pl = list(all_keys)[i]
        # similarity between the current pl and other pls
        dist_dict_temp = {x:-np.inf for x in all_keys}
        for other_pl in all_keys:
            if this_pl != other_pl:
                dist_dict_temp[other_pl] = similarity_dict[(this_pl, other_pl)]
        dist_dict_temp = sorted(dist_dict_temp.items(), key = lambda x:x[1], reverse = True)[:n_neighbors]
        neighbor_dict[this_pl] = [a[0] for a in dist_dict_temp]
        neighbor_sim_dict[this_pl] = [a[1] for a in dist_dict_temp]
    return neighbor_dict, neighbor_sim_dict

def load_neighbor_map(prefix = 'X_train'):
    ''' Loads a pre-calculated neighbor map along with similarity map. '''
    with open(data_path + prefix + "_neighbor_sim_map.pkl", "rb") as f:
        neighbor_dict, neighbor_sim_dict = pickle.load(f)
    return neighbor_dict, neighbor_sim_dict

In [9]:
# # Calculate neighbor_dict, neighbor_sim_dict for the first time 
# neighbor_dict, neighbor_sim_dict = get_neighbor_dict(X_train, similarity_dict)
# print(neighbor_dict[list(neighbor_dict.keys())[0]])
# print(neighbor_sim_dict[list(neighbor_dict.keys())[0]])

# with open(data_path + "X_train_neighbor_sim_map.pkl", "wb") as f:
#     pickle.dump((neighbor_dict, neighbor_sim_dict), f)
    
# Load saved neighbor_dict, neighbor_sim_dict
neighbor_dict, neighbor_sim_dict = load_neighbor_map(prefix = 'X_train')

['835_542', '350_59', '465_645', '546_161', '730_669', '793_316', '827_584', '832_718', '843_654', '890_122', '62_367', '509_938', '33_907', '35_9', '354_189', '375_843', '38_958', '389_392', '401_227', '406_65', '410_772', '42_850', '441_485', '449_295', '45_544', '5_476', '503_429', '505_685', '52_639', '537_312', '540_87', '577_192', '588_255', '591_112', '592_359', '594_976', '598_479', '603_163', '613_638', '624_551', '631_834', '638_907', '640_189', '651_785', '652_334', '654_949', '692_959', '729_320', '779_206', '779_279', '785_448', '795_836', '797_820', '801_341', '805_986', '808_668', '813_75', '815_36', '830_801', '843_948', '845_905', '859_386', '863_360', '866_727', '891_406', '895_755', '897_629', '90_445', '903_928', '928_957', '937_444', '939_638', '948_692', '959_485', '969_284', '396_382', '42_472', '662_934', '817_116', '828_566', '394_459', '80_258', '802_321', '85_722', '37_39', '387_428', '411_644', '512_39', '60_846', '782_393', '836_622', '982_186', '994_218', 

In [10]:
def knn_ranking_algo(X, mode = 'train', n_neighbors = 100, n_preds = 10):    
    ''' The main algorithm that predicts n_preds songs for each of the input playlist. '''
    # Update the neighbor_dict and neighbor_sim_dict based on mode
    neighbor_dict, neighbor_sim_dict = load_neighbor_map(prefix = 'X_'+mode)
    pl_orig_song_lookup = X.copy()
    if mode != 'train':
        for k,v in X_test.items():
            pl_orig_song_lookup[k] = v
    
    predictions = []
    num_runs = len(pl_orig_song_lookup.keys())
    # Loop over all of the 
    for pl_id, contained in pl_orig_song_lookup.items():
        if len(predictions) % (num_runs//100) == 0: print(len(predictions))
        non_contain = [h for h in pop_sorted_tracks if h not in contained]
        # For all the non_contain songs and all neighbors of pl_id, calculates the score
        song_scores = []
        for a_song in non_contain:
            score = 0
            neighbors = neighbor_dict[pl_id]
            for neighbor_ind, neighbor in enumerate(neighbors):
                neighbor_songs = pl_orig_song_lookup[neighbor]
                if a_song in neighbor_songs:
                    score += neighbor_sim_dict[pl_id][neighbor_ind]
            song_scores.append(score)
        # Predict the songs with the highest score
        best_songs = np.array(song_scores).argsort()[-n_preds:][::-1]
        pred = [non_contain[i] for i in best_songs]
        predictions.append(pred)
    return predictions


In [11]:
def overlap_score(tracks, pred_tracks, test_size = 10):
    ''' Computes the overlap score for tracks and pred_tracks. 
        returns #overlap'''
    assert len(tracks) == len(pred_tracks)
    return sum([a in tracks for a in pred_tracks])

def avg_overlap(true_dict, pred):
    ''' Returns the accuracy score given true_label and pred'''
    assert len(true_dict) == len(pred)
    avg_overlap = np.mean([overlap_score(a, b) for a,b in zip(true_dict.values(), pred)])    
    return avg_overlap

In [12]:
predictions = knn_ranking_algo(X_train, n_neighbors = 100, n_preds = 10)
print("Average overlap score is:", avg_overlap(y_train, predictions))

0
74
148
222
296
370
444
518
592
666
740
814
888
962
1036
1110
1184
1258
1332
1406
1480
1554
1628
1702
1776
1850
1924
1998
2072
2146
2220
2294
2368
2442
2516
2590
2664
2738
2812
2886
2960
3034
3108
3182
3256
3330
3404
3478
3552
3626
3700
3774
3848
3922
3996
4070
4144
4218
4292
4366
4440
4514
4588
4662
4736
4810
4884
4958
5032
5106
5180
5254
5328
5402
5476
5550
5624
5698
5772
5846
5920
5994
6068
6142
6216
6290
6364
6438
6512
6586
6660
6734
6808
6882
6956
7030
7104
7178
7252
7326
7400
Average overlap score is: 0.7709086043088452


In [13]:
with open(data_path + "train_data_predictions_knn_ranking.pkl", "wb") as f:
    pickle.dump(predictions, f)